In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix


from collections import defaultdict

from tqdm import tqdm
tqdm.pandas()

import numpy as np


In [4]:
#import spacy.cli
#spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_lg")

In [4]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."
text1= NER(raw_text)

In [5]:
for word in text1.ents:
    print(word.text,word.label_)

The Indian Space Research Organisation ORG
India GPE
Bengaluru GPE
Department of Space ORG
India GPE
ISRO ORG


In [11]:
def ner_extract(raw_text):
  try:
    text1= NER(raw_text)
    lst = [(word.text, word.label_) for word in text1.ents]
    return lst
  except:
    return []

ner_extract(raw_text)

[('The Indian Space Research Organisation', 'ORG'),
 ('India', 'GPE'),
 ('Bengaluru', 'GPE'),
 ('Department of Space', 'ORG'),
 ('India', 'GPE'),
 ('ISRO', 'ORG')]

In [10]:
#news_claim.csv : this file provides a list of news contents that include claim number, group, text, and news source.
#All the data were grouped as follows: 0 as COVID True claims, 1 as COVID Fake claims, 2 as Non-COVID True claims, and 3 as Non-COVID Fake claims.
news_claim = pd.read_csv('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/news_claim/news_claim.csv')
news_claim['NER'] = news_claim.text.progress_apply(lambda x: ner_extract(x))
news_claim

100%|██████████| 1353/1353 [00:14<00:00, 92.47it/s]


,text,source,claim_num,group,NER
0,the (covid 19) cases are going up but it's bec...,politifact,0,1,"[(19, CARDINAL)]"
1,every election year has a disease sars in 200...,politifact,1,1,"[(every election year, DATE), (2004, DATE), (2..."
2,obama ordered cia to train isis jihadists,politifact,2,3,"[(obama, PERSON), (cia, ORG)]"
3,"says george soros said, im going to bring down...",politifact,3,3,"[(george soros, PERSON), (the united states, G..."
4,if you paid rent in march or april they got to...,politifact,4,1,"[(march or, DATE), (april, DATE)]"
...,...,...,...,...,...
1348,says a photo shows patrick mahomes wearing a s...,politifact,1348,3,"[(patrick mahomes, PERSON), (kansas, GPE)]"
1349,nancy pelosi may have just committed a violati...,politifact,1349,3,"[(nancy, PERSON), (18, CARDINAL), (2071, DATE)..."
1350,"says donald trump tweeted, i have it on good a...",politifact,1350,3,"[(donald trump, PERSON), (one, CARDINAL), (dnc..."
1351,says eight iowa counties have more adults regi...,politifact,1351,3,"[(eight, CARDINAL), (iowa, GPE)]"


In [12]:
claim_propagation = pd.read_csv('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation/claim_propagation.csv')
claim_propagation['NER'] = claim_propagation.post_text.progress_apply(lambda x: ner_extract(x))
claim_propagation

100%|██████████| 221253/221253 [42:46<00:00, 86.20it/s]


,tweet_user,tweet_id,like_count,depth,parent_user,create_date,parent_id,retweet_count,post_text,claim_number,group,hashtag,NER
0,64,1260590991641362436,301,0,0,2020-05-13T15:21:27.000Z,0,113,So you left a 69 page guidebook but no PPE or ...,141,1,NaN,"[(69, CARDINAL), (PPE, ORG)]"
1,135,1288771853822103552,194,0,0,2020-07-30T09:42:08.000Z,0,99,More tales from the NHS frontline are emerging...,138,0,NaN,"[(NHS, ORG), (wards &amp, ORG), (yesterday, DA..."
2,290,1251209120528637952,7650,0,0,2020-04-17T18:01:15.000Z,0,4279,DEVELOPING: A crowd of protesters could be see...,30,0,NaN,"[(@GovTimWalz, PERSON), (American, NORP), (Min..."
3,361,1339567071588442113,621,0,0,2020-12-17T13:44:12.000Z,0,231,"Latest we expect on checks, *as of now*, per a...",285,3,NaN,"[(600, MONEY), (600, MONEY), (Rubio, PERSON)]"
4,457,1265997510738358272,959,0,0,2020-05-28T13:25:02.000Z,0,271,Wisconsin election officials agree to send vot...,35,3,NaN,"[(Wisconsin, GPE)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221248,144736,1235937681047080960,0,1,106463,2020-03-06T14:38:00.000Z,1235932698688053249,0,"#WaterIsLife \nremember your vote matters, \nC...",229,2,WaterIsLife#WhenWeAllVote#FlintWaterCrisis,[]
221249,144737,1279815303631798273,1,1,141002,2020-07-05T16:32:00.000Z,1279489733249433600,1,don’t go get tested. i’m sure you will beat so...,131,1,NaN,"[(98%, PERCENT)]"
221250,144738,1242264347025571842,0,1,93248,2020-03-24T01:37:55.000Z,1242257594540056576,1,NO @GOP should even entertain this anti-Americ...,96,0,NaN,"[(anti-American, NORP), (@realDonaldTrump @se..."
221251,144739,1234597235746394118,0,1,56172,2020-03-02T21:51:33.000Z,1234592892896923652,0,I assume if you win it will the begging of the...,173,3,NaN,"[(AMERICAN IMPERIALISM, ORG), (https://t.co/JF..."


In [13]:
origin_data = pd.read_csv('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation/origin_tweet.csv')
origin_data['NER'] = origin_data.post_text.progress_apply(lambda x: ner_extract(x))

origin_data

100%|██████████| 1774/1774 [00:26<00:00, 67.17it/s]


,create_date,tweet_id,similarity,like_count,retweet_count,post_text,claim_number,group,hashtag,tweet_user,NER
0,2020-07-28T14:04:36.000Z,1288113129889517578,0.000000,2823,1050,The latest round of testing from the Miami Mar...,0,1,NaN,85952,"[(the Miami Marlins, ORG), (four, CARDINAL), (..."
1,2020-07-28T03:36:37.000Z,1287955090184105984,0.000000,54,100,Scammers are cold calling Victorians claiming ...,0,1,COVID,5342,"[(Victorians, NORP), (50, MONEY)]"
2,2020-07-27T21:05:06.000Z,1287856562200088581,0.000000,822,428,"“It feels like we’re alone,” Hialeah, FL, Mayo...",0,1,NaN,78787,"[(Hialeah, GPE), (FL, GPE), (Carlos Hernandez,..."
3,2020-07-27T14:54:11.000Z,1287763218933395461,0.000000,36687,14982,The best laid plans of a multi-billion dollar ...,0,1,NaN,80798,"[(multi-billion dollar, MONEY), (a week, DATE)]"
4,2020-04-02T14:37:32.000Z,1245722037869477896,0.404285,13276,4154,NYC’s case fatality rate for Covid19 is now al...,57,1,NaN,80798,"[(NYC, GPE), (almost 3%, PERCENT)]"
...,...,...,...,...,...,...,...,...,...,...,...
1769,2020-12-10T06:10:04.000Z,1336916069336948736,0.445844,2328,345,Struggling after speaking with ⁦@donlemon⁩ ton...,298,1,COVID19,61051,"[(tonight, TIME), (3,000, CARDINAL), (American..."
1770,2020-12-04T15:10:13.000Z,1334877674540572672,0.328623,11593,2454,As the dust settles on Democrats' downballot f...,298,1,NaN,42186,"[(Democrats, NORP), (Democratic, NORP)]"
1771,2020-12-05T11:19:16.000Z,1335181945635004417,0.659882,1436,180,We broke the news on #OutLoudWithGiannoCaldwel...,298,1,OutLoudWithGiannoCaldwell,73973,"[(Senate, ORG), (North Carolina""I, LOC), (Nort..."
1772,2020-12-20T20:31:55.000Z,1340756841454850049,0.390273,171932,2654,I see your tweets and I’m trying to double and...,298,1,NaN,75482,"[(MrBeast Burgers, ORG), (🥺, MONEY)]"


In [14]:
news_claim.to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/news_claim_NER.h5',compression='xz')
claim_propagation.to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/claim_propagation_NER.h5',compression='xz')
origin_data.to_pickle('/content/drive/MyDrive/QE/Study2/FibVID-v0.0.1/dataset/origin_data_NER.h5',compression='xz')